In [19]:
import pandas as pd
import csv
import pickle
import numpy as np
from IPython.display import display
import multiprocessing
from multiprocessing import Pool
import math
from tqdm import tqdm
import time
import traceback
import json

In [20]:
path = "/Users/jay/MSC_WSBDA/MSc_Thesis/Msc_project/Data/"
current_time = 0

In [21]:
def load_pickle_file(pickled_file):
    print(f'Loading data file from {pickled_file}')
    infile = open(pickled_file,'rb')
    unpickled_file = pickle.load(infile)
    print(f'Loaded {len(unpickled_file)} entries')
    infile.close()
    return unpickled_file
          
    
def save_pickle_file(path, data):
    print('Dumping data to path {}'.format(path))
    with open(path, 'wb') as file:
        pickle.dump(data, file)
    print('Finished dumping data to path {}'.format(path))


def mean(numbers):
    return float(sum(numbers)) / max(len(numbers), 1)


def safe_division(x, y):
    if y == 0:
        return 0
    else:
        return x/y




In [22]:
users = load_pickle_file(path+"users.dat")
users.reset_index(drop =True , inplace =True)

Loading data file from /Users/jay/MSC_WSBDA/MSc_Thesis/Msc_project/Data/users.dat
Loaded 5973 entries


In [23]:
network_simulation = pd.DataFrame(columns= ['id','time_lapsed','favourites_count','followers_count','friends_count',
                                'listed_count','statuses_count','source_candidates','source_index','seed_index','generation',
                                 'time_since_seed','user_created_days','normalized_statuses_count','normalized_followers_count',
                                 'normalized_favourites_count','normalized_listed_count','normalized_friends_count'])



In [24]:
network_simulation['id']=users['id']
network_simulation['favourites_count']=users['favourites_count']

network_simulation['followers_count']=users['followers_count']

network_simulation['friends_count']=users['friends_count']

network_simulation['listed_count']=users['listed_count']

network_simulation['statuses_count']=users['statuses_count']

network_simulation['user_created_days']=users['user_created_days']

network_simulation['normalized_statuses_count']=users['normalized_statuses_count']

network_simulation['normalized_followers_count']=users['normalized_followers_count']

network_simulation['normalized_favourites_count']=users['normalized_favourites_count']
network_simulation['normalized_listed_count']=users['normalized_listed_count']

network_simulation['normalized_friends_count']=users['normalized_friends_count']

network_simulation['time_lapsed'] = None
network_simulation['source_index'] = None

network_simulation['seed_index'] = None

network_simulation['generation'] = None

network_simulation['time_since_seed'] = None

In [25]:
friends_list = pd.read_csv(path+"test_1.csv")
friends_list.reset_index(drop =True , inplace =True)

In [26]:
user_ids = set(users['id'])

In [27]:
friends_list['friends'] = friends_list.apply(lambda x: list(set(json.loads(x['friends'])) & user_ids), axis=1)

In [28]:
network_simulation["friends_list"] = friends_list['friends']

In [29]:
followers = pd.read_csv(path+"mr.out",sep='\t',header = None)
followers.reset_index(drop =True , inplace =True)

In [30]:
followers=followers.rename(columns={0:'id'})
followers=followers.rename(columns={1:'followers_list'})

In [31]:
followers["followers_list"] = followers.apply(lambda row: [int(x) for x in json.loads(row["followers_list"])],axis=1)

In [32]:
network_simulation = pd.merge(network_simulation,followers,on='id',how='outer')

In [33]:
users = pd.merge(users,followers,on='id',how='outer')

In [34]:
in_degree = list(users.friends_count)
out_degree = list(users.followers_count)
degree = in_degree + out_degree

In [35]:
def process_data(start_index, end_index,current_time):
    
    features = {
        #Columns which are added for simulation, but they are not used as features for model prediction
        'user_id':[],
        'infected_status':[],
        'infection_time':[],
        'followers_list':[],
        
        #Columns used as features for model prediction
        'UsM_deltaDays': [],
        'UsM_statusesCount': [],
        'UsM_followersCount': [],
        'UsM_favouritesCount': [],
        'UsM_friendsCount': [],
        'UsM_listedCount': [],
        'UsM_normalizedUserStatusesCount': [],
        'UsM_normalizedUserFollowersCount': [],
        'UsM_normalizedUserFavouritesCount': [],
        'UsM_normalizedUserListedCount': [],
        'UsM_normalizedUserFriendsCount': [],          
        'UsM_deltaDays0': [],
        'UsM_statusesCount0': [],
        'UsM_followersCount0': [],
        'UsM_favouritesCount0': [],
        'UsM_friendsCount0': [],
        'UsM_listedCount0': [],
        'UsM_normalizedUserStatusesCount0': [],
        'UsM_normalizedUserFollowersCount0': [],
        'UsM_normalizedUserFavouritesCount0': [],
        'UsM_normalizedUserListedCount0': [],
        'UsM_normalizedUserFriendsCount0': [],
        'UsM_deltaDays-1': [],
        'UsM_statusesCount-1': [],
        'UsM_followersCount-1': [],
        'UsM_favouritesCount-1': [],
        'UsM_friendsCount-1': [],
        'UsM_listedCount-1': [],
        'UsM_normalizedUserStatusesCount-1': [],
        'UsM_normalizedUserFollowersCount-1': [],
        'UsM_normalizedUserFavouritesCount-1': [],
        'UsM_normalizedUserListedCount-1': [],
        'UsM_normalizedUserFriendsCount-1': [],
        # TwM: Tweet metadata
        'TwM_t0': [],
        'TwM_tSeed0': [],
        'TwM_t-1': [],
        'TwM_tSeed-1': [],
        'TwM_tCurrent': [],
        # Nw: Network
        'Nw_degree': [],
        'Nw_inDegree': [],
        'Nw_outDegree': [],
        'Nw_degree0': [],
        'Nw_inDegree0': [],
        'Nw_outDegree0': [],
        'Nw_degree-1': [],
        'Nw_inDegree-1': [],
        'Nw_outDegree-1': [],
        'Nw_degreeSeed0': [],
        'Nw_inDegreeSeed0': [],
        'Nw_outDegreeSeed0': [],
        'Nw_degreeSeed-1': [],
        'Nw_inDegreeSeed-1': [],
        'Nw_outDegreeSeed-1': [],
        # SNw: Spreading Network
        'SNw_nFriendsInfected': [],
        'SNw_friendsInfectedRatio': [],
        'SNw_generation0': [],
        'SNw_generation-1': [],
        'SNw_timeSinceSeed0': [],
        'SNw_timeSinceSeed-1': [],
        'SNw_totalNodesInfected': [],
        'SNw_nodeInfectedCentrality': [],
        'SNw_totalInDegree': [],
        'SNw_totalOutDegree': [],
        'SNw_inDegreeCentrality': [],
        'SNw_inDegreeCentrality0': [],
        'SNw_inDegreeCentrality-1': [],
        'SNw_outDegreeCentrality': [],
        'SNw_outDegreeCentrality0': [],
        'SNw_outDegreeCentrality-1': [],
        'SNw_inDegreeCentralitySeed0':[],
        'SNw_outDegreeCentralitySeed0':[],
        'SNw_inDegreeCentralitySeed-1':[],
        'SNw_outDegreeCentralitySeed-1':[],
        # Stat: Statistical
        'Stat_average_kOut': [],
        'Stat_average_t': [],
        'Stat_average_deltaDays': [],
        'Stat_average_statusesCount': [],
        'Stat_average_followersCount': [],
        'Stat_average_favouritesCount': [],
        'Stat_average_friendsCount': [],
        'Stat_average_listedCount': [],
        'Stat_average_normalizedUserStatusesCount': [],
        'Stat_average_normalizedUserFollowersCount': [],
        'Stat_average_normalizedUserFavouritesCount': [],
        'Stat_average_normalizedUserListedCount': [],
        'Stat_average_normalizedUserFriendsCount': [],                
        'Stat_max_kOut': [],
        'Stat_min_kOut': []
        
    }

    with tqdm(total=len(list(users[start_index: end_index].iterrows()))) as pbar: 
        #print(f"start_index:{start_index}, end_index: {end_index}")
        for index, user_row in users[start_index: end_index].iterrows():
           
            #if len(user_row['source_candidates']) > 0 and ((user_row['source_index'] is not None) or math.isnan(user_row['time_lapsed'])):
            source_candidates = None
            features['user_id'].append(user_row['id'])
            features['infected_status'].append(False)
            features['infection_time'].append(None)
            #print(f"user_row['followers_list']:{user_row['followers_list']}")
            features['followers_list'].append(user_row['followers_list'])
            #print("b")
            features['UsM_deltaDays'].append(user_row['user_created_days'])
            features['UsM_statusesCount'].append(user_row['statuses_count'])
            features['UsM_followersCount'].append(user_row['followers_count'])
            features['UsM_favouritesCount'].append(user_row['favourites_count'])
            features['UsM_friendsCount'].append(user_row['friends_count'])
            features['UsM_listedCount'].append(user_row['listed_count'])
            features['UsM_normalizedUserStatusesCount'].append(user_row['normalized_statuses_count'])
            features['UsM_normalizedUserFollowersCount'].append(user_row['normalized_followers_count'])
            features['UsM_normalizedUserFavouritesCount'].append(user_row['normalized_favourites_count'])
            features['UsM_normalizedUserListedCount'].append(user_row['normalized_listed_count'])
            features['UsM_normalizedUserFriendsCount'].append(user_row['normalized_friends_count'])    

            sources = None
           # sources
            #if len(sources) > 0:


                # Assign the values here to save computation
            first_source_index = None
            first_source_row = None
            first_source_seed_row = None

#                     start_bar = int(first_source_row.time_lapsed / interval) + 1
#                     # + 1 to get more infected data
#                     end_bar = min((user_row['time_lapsed'] / interval) + 2, (24*60/interval)) 
#                     bars = list(np.arange(start_bar * interval, end_bar * interval, interval))
#                     number_of_bars = len(bars)



            #for current_time in bars:
                # all sources up to the current time

#             sources_dataframe = users.loc[sources]
#             degreeList = list(degree[i] for i in sources)
#             inDegreeList = list(in_degree[i] for i in sources)
#             outDegreeList = list(out_degree[i] for i in sources)
#             degreeList = list(users.loc[i, 'followers_count'] + users.loc[i, 'friends_count']  for i in sources)
#             timeList = [current_time - users.loc[x].time_lapsed for x in sources]


#             last_source_index = None                        
#             last_source_row = users.loc[last_source_index]
#             last_source_seed_row = users.loc[last_source_row['seed_index']]

            #Extraction
            #Columns which are added for simulation, but they are not used as features for model prediction

            #usr_index = user_row.index.values
            usr_index = index
            #print(f"usr_index: {usr_index}")

            #print(f"user_row['time_lapsed']:{user_row['time_lapsed']}")

            network_simulation.at[usr_index,'source_candidates'] = sources

            # UsM: User metadata                    

            features['UsM_deltaDays0'].append(None)
            features['UsM_statusesCount0'].append(None)
            features['UsM_followersCount0'].append(None)
            features['UsM_favouritesCount0'].append(None)
            features['UsM_friendsCount0'].append(None)
            features['UsM_listedCount0'].append(None)
            features['UsM_normalizedUserStatusesCount0'].append(None)
            features['UsM_normalizedUserFollowersCount0'].append(None)
            features['UsM_normalizedUserFavouritesCount0'].append(None)
            features['UsM_normalizedUserListedCount0'].append(None)
            features['UsM_normalizedUserFriendsCount0'].append(None)
            features['UsM_deltaDays-1'].append(None)
            features['UsM_statusesCount-1'].append(None)
            features['UsM_followersCount-1'].append(None)
            features['UsM_favouritesCount-1'].append(None)
            features['UsM_friendsCount-1'].append(None)
            features['UsM_listedCount-1'].append(None)
            features['UsM_normalizedUserStatusesCount-1'].append(None)
            features['UsM_normalizedUserFollowersCount-1'].append(None)
            features['UsM_normalizedUserFavouritesCount-1'].append(None)
            features['UsM_normalizedUserListedCount-1'].append(None)
            features['UsM_normalizedUserFriendsCount-1'].append(None) 
            # TwM: Tweet metadata
            features['TwM_t0'].append(None)
            features['TwM_tSeed0'].append(None)
            features['TwM_t-1'].append(None)
            features['TwM_tSeed-1'].append(None)
            features['TwM_tCurrent'].append(None)
            # Nw: Network
            features['Nw_degree'].append(degree[index])
            features['Nw_inDegree'].append(in_degree[index])
            features['Nw_outDegree'].append(out_degree[index])
            features['Nw_degree0'].append(None)
            features['Nw_inDegree0'].append(None)
            features['Nw_outDegree0'].append(None)
            features['Nw_degree-1'].append(None)
            features['Nw_inDegree-1'].append(None)
            features['Nw_outDegree-1'].append(None)
            features['Nw_degreeSeed0'].append(None)
            features['Nw_inDegreeSeed0'].append(None)
            features['Nw_outDegreeSeed0'].append(None)
            features['Nw_degreeSeed-1'].append(None)
            features['Nw_inDegreeSeed-1'].append(None)
            features['Nw_outDegreeSeed-1'].append(None)
            # SNw: Spreading Network
            features['SNw_nFriendsInfected'].append(0)
            features['SNw_friendsInfectedRatio'].append(0)
            features['SNw_generation0'].append(None)
            features['SNw_generation-1'].append(None)
            features['SNw_timeSinceSeed0'].append(None)
            features['SNw_timeSinceSeed-1'].append(None)

            #infected_dataframe = users[users.time_lapsed <= current_time]
#             total_nodes_infected = infected_dataframe.shape[0]
#             total_in_degree = sum(infected_dataframe.friends_count)
#             total_out_degree = sum(infected_dataframe.followers_count)

            features['SNw_totalNodesInfected'].append(0)
            features['SNw_nodeInfectedCentrality'].append(None)
            features['SNw_totalInDegree'].append(0)
            features['SNw_totalOutDegree'].append(0)
            features['SNw_inDegreeCentrality'].append(None)
            features['SNw_inDegreeCentrality0'].append(None)
            features['SNw_inDegreeCentrality-1'].append(None)
            features['SNw_outDegreeCentrality'].append(None)
            features['SNw_outDegreeCentrality0'].append(None)
            features['SNw_outDegreeCentrality-1'].append(None)
            features['SNw_inDegreeCentralitySeed0'].append(None)
            features['SNw_outDegreeCentralitySeed0'].append(None)
            features['SNw_inDegreeCentralitySeed-1'].append(None)
            features['SNw_outDegreeCentralitySeed-1'].append(None)
            # Stat: Statistical
            features['Stat_average_kOut'].append(None)
            features['Stat_average_t'].append(None)
            features['Stat_average_deltaDays'].append(None)
            features['Stat_average_statusesCount'].append(None)
            features['Stat_average_followersCount'].append(None)
            features['Stat_average_favouritesCount'].append(None)
            features['Stat_average_friendsCount'].append(None)
            features['Stat_average_listedCount'].append(None)
            features['Stat_average_normalizedUserStatusesCount'].append(None)
            features['Stat_average_normalizedUserFollowersCount'].append(None)
            features['Stat_average_normalizedUserFavouritesCount'].append(None)
            features['Stat_average_normalizedUserListedCount'].append(None)
            features['Stat_average_normalizedUserFriendsCount'].append(None)
            features['Stat_max_kOut'].append(None)
            features['Stat_min_kOut'].append(None)  
            processed_dataframe = pd.DataFrame(features)                 
            pbar.update(1)
    return processed_dataframe

In [36]:
start_index = 0
end_index = 5973

features = process_data(start_index, end_index,current_time)
save_pickle_file(path+"0_hrs_data.pkl",features)
save_pickle_file(path+"network_simulation_0_hrs.pkl",network_simulation)
        

100%|██████████| 5973/5973 [04:30<00:00, 22.07it/s]

Dumping data to path /Users/jay/MSC_WSBDA/MSc_Thesis/Msc_project/Data/0_hrs_data.pkl
Finished dumping data to path /Users/jay/MSC_WSBDA/MSc_Thesis/Msc_project/Data/0_hrs_data.pkl
Dumping data to path /Users/jay/MSC_WSBDA/MSc_Thesis/Msc_project/Data/network_simulation_0_hrs.pkl
Finished dumping data to path /Users/jay/MSC_WSBDA/MSc_Thesis/Msc_project/Data/network_simulation_0_hrs.pkl
